In [26]:
from yargy import rule, or_, Parser
from yargy.predicates import gram

FIRST = gram('Name')
LAST = gram('Surn')
MIDDLE = gram('Patr')
ABBR = gram('Abbr')

NAME = or_(
    rule(FIRST),
    rule(LAST),
    rule(FIRST, LAST),
    rule(LAST, FIRST),
    rule(FIRST, MIDDLE, LAST),
    rule(LAST, FIRST, MIDDLE),
    rule(ABBR, '.', ABBR, '.', LAST),
    rule(LAST, ABBR, '.', ABBR, '.'),
)

parser = Parser(NAME)
text = '''
Хочу поблагодарить учителей моего, теперь уже бывшего, одиннадцатиклассника:
Бушуева Вячеслава Владимировича и Бушуеву Веру Константиновну. 
'''
for match in parser.findall(text):
    start, stop = match.span
    print(text[start:stop])

Бушуева Вячеслава Владимировича
Бушуеву Веру Константиновну


In [12]:
from yargy.interpretation import fact

Name = fact(
    'Name',
    ['first', 'middle', 'last']
)

FIRST = gram('Name').interpretation(
    Name.first.inflected()
)
LAST = gram('Surn').interpretation(
    Name.last.inflected()
)
MIDDLE = gram('Patr').interpretation(
    Name.middle.inflected()
)
ABBR = gram('Abbr')
FIRST_ABBR = ABBR.interpretation(
    Name.first.custom(str.lower)
)
MIDDLE_ABBR = ABBR.interpretation(
    Name.middle.custom(str.lower)
)

NAME = or_(
    rule(FIRST),
    rule(LAST),
    rule(FIRST, LAST),
    rule(FIRST_ABBR, '.', MIDDLE_ABBR, '.', LAST),
    rule(LAST, FIRST, MIDDLE),
    rule(FIRST, LAST, MIDDLE)
).interpretation(
    Name
)
    
parser = Parser(NAME)
text = '''
Хочу поблагодарить учителей моего, теперь уже бывшего, одиннадцатиклассника:
Бушуева Вячеслава Владимировича и Бушуеву Веру Константиновну. 
'''
for match in parser.findall(text):
    print(match.fact)

Name(first='вячеслав', middle='владимирович', last='бушуев')
Name(first='вера', middle='константиновна', last='бушуев')


In [27]:
from yargy.relations import gnc_relation

gnc = gnc_relation()

FIRST = gram('Name').interpretation(
    Name.first.inflected()
).match(gnc)

LAST = gram('Surn').interpretation(
    Name.last.inflected()
).match(gnc)

MIDDLE = gram('Patr').interpretation(
    Name.middle.inflected()
).match(gnc)

ABBR = gram('Abbr')
FIRST_ABBR = ABBR.interpretation(
    Name.first.custom(str.lower)
)
MIDDLE_ABBR = ABBR.interpretation(
    Name.middle.custom(str.lower)
)

NAME = or_(
    rule(FIRST),
    rule(LAST),
    rule(FIRST, LAST),
    rule(FIRST_ABBR, '.', MIDDLE_ABBR, '.', LAST),
    rule(LAST, FIRST, MIDDLE),
    rule(FIRST, LAST, MIDDLE)
).interpretation(
    Name
)
    
parser = Parser(NAME)
text = '''
Хочу поблагодарить учителей моего, теперь уже бывшего, одиннадцатиклассника:
Бушуева Вячеслава Владимировича и Бушуеву Веру Константиновну. 
'''
for match in parser.findall(text):
    print(match.fact)

Name(first='вячеслав', middle='владимирович', last='бушуев')
Name(first='вера', middle='константиновна', last='бушуева')


In [28]:
from natasha import NamesExtractor

extractor = NamesExtractor()

for match in extractor(text):
    print(match.fact)

Name(first='вячеслав', middle='владимирович', last='бушуев', nick=None)
Name(first='вера', middle='константиновна', last='бушуева', nick=None)


In [25]:
from natasha.markup import show_markup

text = '''
Байкеры мотоклуба «Ночные волки» совместно с главой Чечни Рамзаном Кадыровым
совершили мотопробег, посвященный дню рождения президента РФ Владимира Путина.
Об этом сообщил в «ВКонтакте» сам Кадыров. «В среду вечером мы организовали
мотопробег, посвященный дню рождения президента России Владимира Путина.

В США в возрасте 92 лет скончалась негритянка Роза Паркс, ставшая символом
правозашитного движения в США, сообщает Reuters.

В составе "Эдмонтона" голы забили Райан Джонс, Том Гилберт, а также
Райан Смит, который поразил пустые ворота.

Следом за Свифт идут Кэти Перри (219 миллионов), Селена Гомес
(205 миллионов) и Рианна (190 миллионов)
'''

matches = extractor(text)
spans = [_.span for _ in matches]
show_markup(text, spans)
# for match in matches:
#     display(match.fact)


Байкеры мотоклуба «Ночные волки» совместно с главой Чечни [[Рамзаном Кадыровым]]
совершили мотопробег, посвященный дню рождения президента РФ [[Владимира Путина]].
Об этом сообщил в «ВКонтакте» сам [[Кадыров]]. «В среду вечером мы организовали
мотопробег, посвященный дню рождения президента России [[Владимира Путина]].

В США в возрасте 92 лет скончалась негритянка [[Роза]] [[Паркс]], ставшая символом
правозашитного движения в США, сообщает Reuters.

В составе "Эдмонтона" голы забили [[Райан Джонс]], Том [[Гилберт]], а также
[[Райан Смит]], который поразил пустые ворота.

Следом за Свифт идут [[Кэти]] [[Перри]] (219 миллионов), Селена [[Гомес]]
(205 миллионов) и [[Рианна]] (190 миллионов)



In [20]:
from natasha import AddressExtractor

text = '''
353691, Ейск г, Коммунистическая ул, д.85/6, оф. 10 , («Ейск Коммунистическая_2349_С

типография: г.Люберцы, ул. Инициативная, д.5

601440, г. Вязники, Владимирская область, микрорайон Дечинский, д. 11.
'''

extractor = AddressExtractor()
matches = extractor(text)
display(matches)
for match in matches:
    display(match.fact)

Matches('\n353691, Ейск г, Коммунистическая ул, д.85/6, оф. 10 , («Ейск Коммунистическая_2349_С\n\nтипография: г.Люберцы, ул. Инициативная, д.5\n\n601440, г. Вязники, Владимирская область, микрорайон Дечинский, д. 11.\n',
        [Match([MorphTagToken('Коммунистическая',
                              [17, 33),
                              'RU',
                              'I',
                              [Form('коммунистический',
                                    Grams(ADJF,femn,nomn,sing))]),
                MorphTagToken('ул',
                              [34, 36),
                              'RU',
                              'O',
                              [Form('ул', Grams(UNKN))]),
                TagToken(',', [36, 37), 'PUNCT', 'O'),
                MorphTagToken('д',
                              [38, 39),
                              'RU',
                              'O',
                              [Form('далее', Grams(ADVB,Abbr))]),
                TagToken('.', [39, 40), 'PUNCT', 'O'),
                TagToken('85', [40, 42), 'INT', 'O'),
                TagToken('/', [42, 43), 'PUNCT', 'O'),
                TagToken('6', [43, 44), 'INT', 'O'),
                TagToken(',', [44, 45), 'PUNCT', 'O'),
                MorphTagToken('оф',
                              [46, 48),
                              'RU',
                              'O',
                              [Form('оф', Grams(UNKN))]),
                TagToken('.', [48, 49), 'PUNCT', 'O'),
                TagToken('10', [50, 52), 'INT', 'O')],
               [17, 52)),
         Match([MorphTagToken('г',
                              [99, 100),
                              'RU',
                              'O',
                              [Form('год',
                                    Grams(Abbr,Fixd,NOUN,inan,masc,nomn,sing)),
                               Form('год',
                                    Grams(Abbr,Fixd,NOUN,gent,inan,masc,sing)),
                               Form('год',
                                    Grams(Abbr,Fixd,NOUN,datv,inan,masc,sing)),
                               Form('год',
                                    Grams(Abbr,Fixd,NOUN,accs,inan,masc,sing)),
                               Form('год',
                                    Grams(Abbr,Fixd,NOUN,ablt,inan,masc,sing)),
                               Form('год',
                                    Grams(Abbr,Fixd,NOUN,inan,loct,masc,sing)),
                               Form('год',
                                    Grams(Abbr,Fixd,NOUN,inan,masc,nomn,plur)),
                               Form('год',
                                    Grams(Abbr,Fixd,NOUN,gent,inan,masc,plur)),
                               Form('год',
                                    Grams(Abbr,Fixd,NOUN,datv,inan,masc,plur)),
                               Form('год',
                                    Grams(Abbr,Fixd,NOUN,accs,inan,masc,plur)),
                               Form('год',
                                    Grams(Abbr,Fixd,NOUN,ablt,inan,masc,plur)),
                               Form('год',
                                    Grams(Abbr,Fixd,NOUN,inan,loct,masc,plur))]),
                TagToken('.', [100, 101), 'PUNCT', 'O'),
                MorphTagToken('Люберцы',
                              [101, 108),
                              'RU',
                              'O',
                              [Form('люберцы',
                                    Grams(GNdr,Geox,NOUN,Pltm,inan,nomn,plur)),
                               Form('люберцы',
                                    Grams(GNdr,Geox,NOUN,Pltm,accs,inan,plur))]),
                TagToken(',', [108, 109), 'PUNCT', 'O'),
                MorphTagToken('ул',
                              [110, 112),
                              'RU',
                              'O',
                              [Form('ул', Grams(UNKN))]),
                TagToken('.', [112, 113), 'PUNCT', 'O

Address(parts=[Street(name='Коммунистическая',
                      type='улица'), Building(number='85/6',
                        type='дом'), Room(number='10',
                    type='офис')])

Address(parts=[Settlement(name='Люберцы',
                          type='город'), Street(name='Инициативная',
                      type='улица'), Building(number='5',
                        type='дом')])